In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler


In [2]:
import preprocess
real_instances, simulated_instances, drawn_instances = preprocess.get_all_labels_and_files(by_label=False)

In [3]:
def count(instances):
    c = {}
    for k, lst in instances.items():
        c[k] = {}
        c[k]['normal'] = sum([label==0 for (label, _) in lst])
        c[k]['abnormal'] = len(lst) -  c[k]['normal']
    return c

count(real_instances)

{'WELL-00002': {'normal': 210, 'abnormal': 118},
 'WELL-00008': {'normal': 57, 'abnormal': 0},
 'WELL-00007': {'normal': 2, 'abnormal': 10},
 'WELL-00001': {'normal': 94, 'abnormal': 39},
 'WELL-00005': {'normal': 81, 'abnormal': 38},
 'WELL-00006': {'normal': 115, 'abnormal': 5},
 'WELL-00004': {'normal': 12, 'abnormal': 46},
 'WELL-00003': {'normal': 26, 'abnormal': 3},
 'WELL-00018': {'normal': 0, 'abnormal': 1},
 'WELL-00020': {'normal': 0, 'abnormal': 1},
 'WELL-00019': {'normal': 0, 'abnormal': 1},
 'WELL-00021': {'normal': 0, 'abnormal': 1},
 'WELL-00010': {'normal': 0, 'abnormal': 84},
 'WELL-00014': {'normal': 0, 'abnormal': 52},
 'WELL-00011': {'normal': 0, 'abnormal': 13},
 'WELL-00012': {'normal': 0, 'abnormal': 2},
 'WELL-00013': {'normal': 0, 'abnormal': 1},
 'WELL-00009': {'normal': 0, 'abnormal': 1},
 'WELL-00015': {'normal': 0, 'abnormal': 2},
 'WELL-00016': {'normal': 0, 'abnormal': 4},
 'WELL-00017': {'normal': 0, 'abnormal': 6}}

## Select wells with both normal and abnormal data

In [9]:
filtered_wells = ['WELL-00002', 'WELL-00001', 'WELL-00005',
 'WELL-00006',  'WELL-00004','WELL-00003',]


In [11]:
from tqdm import tqdm

def load_real_instances(filtered_wells, max_instances_per_type = 20):
    df_map = {}
    for well in tqdm(filtered_wells):
        df_normal = []
        df_ab = []
        for (label, fp) in real_instances[well]:
            if len(df_normal) >= max_instances_per_type and len(df_ab) >= max_instances_per_type:
                break

            if label == 0 and len(df_normal) < max_instances_per_type:
                df = pd.read_csv(fp)
                df['start'] = 0
                df.loc[0, 'start'] = 1
                df_normal.append(df)
            elif label != 0 and len(df_ab) < max_instances_per_type:
                df = pd.read_csv(fp)
                df['start'] = 0
                df.loc[0, 'start'] = 1
                df_ab.append(df)

        df_map[well] = {}
        df_map[well]['normal'] = pd.concat(df_normal).reset_index() if df_normal else None
        df_map[well]['abnormal'] = pd.concat(df_ab).reset_index()  if df_ab else None
    return df_map

df_map = load_real_instances(filtered_wells)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:09<00:00,  1.66s/it]


In [16]:
pd.read_csv('/Users/suchanuchpiriyasatit/Documents/Tsinghua/Learning from Data/GDN/raw_data/6/SIMULATED_00009.csv')[
    'class'
].value_counts()

6      17999
106     7200
0       1800
Name: class, dtype: int64

In [8]:
for well in filtered_wells:
    print(f'{well}: normal')
    df = df_map[well]['normal']
    display(df.describe())
    print(f'{well}: abnormal')
    df = df_map[well]['abnormal']
    display(df.describe())

        

WELL-00002: normal


,index,P-PDG,P-TPT,T-TPT,P-MON-CKP,T-JUS-CKP,P-JUS-CKGL,T-JUS-CKGL,QGL,class,start
count,357607.000000,357607.0,3.576070e+05,357607.000000,3.576070e+05,357607.000000,3.576070e+05,0.0,357607.0,357607.0,357607.000000
mean,8939.814117,0.0,8.363871e+06,117.128480,1.673102e+06,74.474361,2.322565e+06,NaN,0.0,0.0,0.000056
std,5161.859885,0.0,3.483591e+05,0.583021,2.386120e+05,2.083131,7.090841e+03,NaN,0.0,0.0,0.007478
min,0.000000,0.0,7.161897e+06,114.996900,9.447082e+05,66.716670,2.307518e+06,NaN,0.0,0.0,0.000000
25%,4470.000000,0.0,8.119477e+06,116.724700,1.498730e+06,73.266515,2.317906e+06,NaN,0.0,0.0,0.000000
50%,8940.000000,0.0,8.426860e+06,117.185300,1.656784e+06,74.949440,2.322125e+06,NaN,0.0,0.0,0.000000
75%,13410.000000,0.0,8.615556e+06,117.559000,1.838063e+06,76.055820,2.325066e+06,NaN,0.0,0.0,0.000000
max,17968.000000,0.0,9.017208e+06,118.629600,2.553461e+06,78.679230,2.340199e+06,NaN,0.0,0.0,1.000000


WELL-00002: abnormal


,index,P-PDG,P-TPT,T-TPT,P-MON-CKP,T-JUS-CKP,P-JUS-CKGL,T-JUS-CKGL,QGL,class,start
count,172953.000000,172953.0,1.729530e+05,172953.000000,1.729530e+05,172953.000000,1.729530e+05,0.0,172953.0,172356.000000,172953.000000
mean,5211.194255,0.0,1.528682e+07,118.058177,5.842944e+06,148.205576,4.520182e+06,NaN,0.0,9.696552,0.000116
std,4543.448486,0.0,1.454005e+06,0.526733,2.053958e+06,40.237557,1.826329e+05,NaN,0.0,24.471857,0.010753
min,0.000000,0.0,1.258818e+07,117.340600,1.629007e+06,80.851490,4.039072e+06,NaN,0.0,0.000000,0.000000
25%,2161.000000,0.0,1.340611e+07,117.591900,4.252456e+06,84.236390,4.383407e+06,NaN,0.0,4.000000,0.000000
50%,4323.000000,0.0,1.614649e+07,117.856600,7.052615e+06,173.096100,4.474512e+06,NaN,0.0,4.000000,0.000000
75%,6485.000000,0.0,1.628783e+07,118.629600,7.335128e+06,173.096100,4.670248e+06,NaN,0.0,4.000000,0.000000
max,24369.000000,0.0,1.715663e+07,119.137400,8.192052e+06,173.096100,4.878340e+06,NaN,0.0,106.000000,1.000000


WELL-00001: normal


...

,index,P-PDG,P-TPT,T-TPT,P-MON-CKP,T-JUS-CKP,P-JUS-CKGL,T-JUS-CKGL,QGL,class,start
count,127870.000000,127826.0,1.277040e+05,127726.000000,1.278700e+05,127708.000000,0.0,0.0,0.0,127731.000000,127870.000000
mean,3462.429405,0.0,1.718782e+07,106.553071,9.129116e+06,68.607330,NaN,NaN,NaN,5.316626,0.000156
std,2081.892489,0.0,1.099137e+06,0.084417,9.772114e+05,1.331411,NaN,NaN,NaN,12.027156,0.012505
min,0.000000,0.0,1.324251e+07,106.455300,5.244155e+06,65.600140,NaN,NaN,NaN,0.000000,0.000000
25%,1625.000000,0.0,1.618616e+07,106.520100,8.511879e+06,67.921190,NaN,NaN,NaN,4.000000,0.000000
50%,3398.000000,0.0,1.775544e+07,106.541000,9.545817e+06,68.334210,NaN,NaN,NaN,4.000000,0.000000
75%,5268.000000,0.0,1.777529e+07,106.563300,9.683469e+06,68.878692,NaN,NaN,NaN,4.000000,0.000000
max,7188.000000,0.0,1.784361e+07,107.017800,9.852484e+06,73.840000,NaN,NaN,NaN,106.000000,1.000000


WELL-00003: normal


,index,P-PDG,P-TPT,T-TPT,P-MON-CKP,T-JUS-CKP,P-JUS-CKGL,T-JUS-CKGL,QGL,class,start
count,356861.000000,3.548320e+05,3.548320e+05,354819.000000,3.564200e+05,0.0,0.0,0.0,0.0,356861.0,356861.000000
mean,8921.308507,3.154497e+07,1.685051e+07,110.652162,9.588333e+06,NaN,NaN,NaN,NaN,0.0,0.000056
std,5151.341069,2.954163e+05,2.738489e+05,0.247736,4.100193e+05,NaN,NaN,NaN,NaN,0.0,0.007486
min,0.000000,3.123479e+07,1.658034e+07,109.938100,9.110818e+06,NaN,NaN,NaN,NaN,0.0,0.000000
25%,4460.000000,3.130786e+07,1.665730e+07,110.525000,9.276197e+06,NaN,NaN,NaN,NaN,0.0,0.000000
50%,8921.000000,3.139216e+07,1.670381e+07,110.776900,9.363394e+06,NaN,NaN,NaN,NaN,0.0,0.000000
75%,13382.000000,3.175347e+07,1.702790e+07,110.854900,9.848848e+06,NaN,NaN,NaN,NaN,0.0,0.000000
max,17974.000000,3.222495e+07,1.751426e+07,110.952800,1.059748e+07,NaN,NaN,NaN,NaN,0.0,1.000000


WELL-00003: abnormal


,index,P-PDG,P-TPT,T-TPT,P-MON-CKP,T-JUS-CKP,P-JUS-CKGL,T-JUS-CKGL,QGL,class,start
count,21610.000000,2.161000e+04,2.161000e+04,21610.000000,2.161000e+04,0.0,0.0,0.0,0.0,21253.000000,21610.000000
mean,3826.153447,3.352345e+07,1.059655e+07,49.373122,3.748674e+06,NaN,NaN,NaN,NaN,72.171740,0.000139
std,2435.289697,1.166157e+06,3.973236e+06,40.431130,4.073438e+06,NaN,NaN,NaN,NaN,46.306226,0.011782
min,0.000000,3.151992e+07,4.860626e+06,6.709711,-8.317492e+03,NaN,NaN,NaN,NaN,0.000000,0.000000
25%,1800.250000,3.256129e+07,7.881582e+06,16.329910,-8.100597e+03,NaN,NaN,NaN,NaN,0.000000,0.000000
50%,3601.000000,3.401390e+07,8.952152e+06,29.672855,1.977000e+06,NaN,NaN,NaN,NaN,102.000000,0.000000
75%,5561.000000,3.435154e+07,1.663634e+07,101.379475,9.051954e+06,NaN,NaN,NaN,NaN,102.000000,0.000000
max,9484.000000,3.496202e+07,1.688088e+07,113.485400,9.173318e+06,NaN,NaN,NaN,NaN,102.000000,1.000000


In [16]:
def normalize(df_train, df_test, cols):
    
    scaler = MinMaxScaler()
    normalizer = MinMaxScaler(feature_range=(0, 1)).fit(df_train[cols])
    norm_train = normalizer.transform(df_train[cols])
    norm_test = normalizer.transform(df_test[cols])
    
    def to_df(norm_data, df):
        ndf = pd.DataFrame(norm_data, columns=cols)
        ndf['start'] = df.start
        ndf['class'] = df['class']
        ndf['timestamp'] = df['timestamp']
        return ndf
    
    ndf_train = to_df(norm_train, df_train)
    ndf_test = to_df(norm_test, df_test)
    
    return ndf_train, ndf_test



In [13]:
from visualization import plot_by_instances

In [18]:
df_map['WELL-00001']['normal'].columns

Index(['index', 'timestamp', 'P-PDG', 'P-TPT', 'T-TPT', 'P-MON-CKP',
       'T-JUS-CKP', 'P-JUS-CKGL', 'T-JUS-CKGL', 'QGL', 'class', 'start'],
      dtype='object')

In [19]:
cols = ['P-PDG', 'P-TPT', 'T-TPT', 'P-MON-CKP',
       'T-JUS-CKP', 'P-JUS-CKGL']
df_norm, df_ab_norm = normalize(df_map['WELL-00001']['normal'], df_map['WELL-00001']['abnormal'], cols=cols)

In [36]:
cols

['P-PDG', 'P-TPT', 'T-TPT', 'P-MON-CKP', 'T-JUS-CKP', 'P-JUS-CKGL']

In [29]:
df = df_map['WELL-00001']['normal']
df = df.dropna(subset=cols+['class'])
df_ab = df_map['WELL-00001']['abnormal']
df_ab = df_ab.dropna(subset=cols+['class'])
df_ab['class'] = df_ab['class'].apply(lambda x: 1 if x!=0 else 0)

/Users/suchanuchpiriyasatit/.pyenv/versions/3.7.9/envs/lfd-gdn/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [34]:
df_ab[df_ab.start==1].index

Int64Index([     0,  44150,  56655,  63845,  71015,  78173,  85353,  92431,
             99621, 106791, 113950, 121109, 128294, 135475, 142650, 149830,
            157015, 167553, 174702, 181877],
           dtype='int64')

In [35]:
df_ab.iloc[:44150].reset_index(drop=True).to_csv(f'data/WELL-00001-updated/test.csv', index=True)

In [33]:
df.reset_index(drop=True).to_csv(f'data/WELL-00001-updated/train.csv', index=True)

In [ ]:
df.reset_index(drop=True).to_csv(f'data/WELL-00003/test.csv', index=True)

In [1]:
# plot_by_instances(df_norm)

In [2]:
# plot_by_instances(df_ab_norm)

In [3]:
# plot_by_instances(df_ab_norm.iloc[:1000])

In [25]:
cols = [ 'P-PDG', 'P-TPT', 'T-TPT', 'P-MON-CKP',
       'T-JUS-CKP', 'P-JUS-CKGL']
df_norm, df_ab_norm = normalize(df_map['WELL-00006']['normal'], df_map['WELL-00006']['abnormal'], cols=cols)

In [4]:
# plot_by_instances(df_norm)

In [5]:
# plot_by_instances(df_ab_norm)